In [ ]:
import numpy as np
import glob
import cv2
import tensorflow as tf

In [ ]:
#数据处理加四类标签，图像resize成200X200大小
data_dir = r'C:/Users/dearw/Desktop/TF/WDSC/dataset/'
classes = ['broadleaf', 'grass', 'soil', 'soybean']

num_file = 1000
num_data =num_file*len(classes)
Y = np.zeros((num_data,4))
X = np.ndarray(shape=(num_data,200,200,3), dtype=np.float64)

for i,cls in enumerate(classes):
    for j in range(1000):
        path = data_dir+cls+'/'+str(j+1)+'.tif'
        temp = cv2.imread(path,1)
        temp = cv2.resize(temp,(200,200))
        if i==0:
            X[4*j] = temp
            Y[4*j,i] = 1
        elif i==1:
            X[4*j+1] = temp
            Y[4*j+1,i] = 1
        elif i==2:
            X[4*j+2] = temp
            Y[4*j+2,i] = 1
        else:
            X[4*j+3] = temp
            Y[4*j+3,i] = 1
    print('note')
X = X/255;
print('finish')

In [ ]:
#img = X[38]
#cv2.imshow("image-type", img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
Train_data = X[0:3600,:,:,:]
Train_label = Y[0:3600,:]
Test_data = X[3600:4000,:,:,:]
Test_label = Y[3600:4000,:]
print(Train_label.shape)

In [ ]:
#分类网络
#批次大小
batch_size = 50
#计算一共有多少个批次
num_size = 3600
n_batch = num_size // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]`
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#命名空间
#定义两个placeholder
x = tf.placeholder(tf.double,[None,200,200,3])
y = tf.placeholder(tf.double,[None,4])

#改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]`
#x_image = tf.reshape(x,[-1,200,200,3])


#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([3,3,3,16])#3*3的采样窗口，16个卷积核从1个平面抽取特征
b_conv1 = bias_variable([16])#每一个卷积核一个偏置值
W_conv1 = tf.cast(W_conv1,tf.double)
b_conv1 = tf.cast(b_conv1,tf.double)

#把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
conv2d_1 = conv2d(x,W_conv1) + b_conv1
h_conv1 = tf.nn.relu(conv2d_1)
h_pool1 = max_pool_2x2(h_conv1)#进行max-pooling

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([3,3,16,32])#3*3的采样窗口，32个卷积核从16个平面抽取特征
b_conv2 = bias_variable([32])#每一个卷积核一个偏置值
W_conv2 = tf.cast(W_conv2,tf.double)
b_conv2 = tf.cast(b_conv2,tf.double)

#把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
conv2d_2 = conv2d(h_pool1,W_conv2) + b_conv2
h_conv2 = tf.nn.relu(conv2d_2)
h_pool2 = max_pool_2x2(h_conv2)#进行max-pooling

#200*200的图片第一次卷积后还是200*200，第一次池化后变为100*100
#第二次卷积后为100*100，第二次池化后变为了50*50
#进过上面操作后得到64张50*50的平面

#初始化第一个全连接层的权值
W_fc1 = weight_variable([50*50*32,1024])#上一层有3*3*64个神经元，全连接层有1024个神经元
b_fc1 = bias_variable([1024])#1024个节点
W_fc1 = tf.cast(W_fc1,tf.double)
b_fc1 = tf.cast(b_fc1,tf.double)

#把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,50*50*32])
#求第一个全连接层的输出
wx_plus_b1 = tf.matmul(h_pool2_flat,W_fc1) + b_fc1
h_fc1 = tf.nn.relu(wx_plus_b1)

#keep_prob用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.double)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层
W_fc2 = weight_variable([1024,4])
b_fc2 = bias_variable([4])
W_fc2 = tf.cast(W_fc2,tf.double)
b_fc2 = tf.cast(b_fc2,tf.double)
wx_plus_b2 = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
prediction = tf.nn.softmax(wx_plus_b2)

#交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

#求准确率
#结果存放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))#argmax返回一维张量中最大的值所在的位置

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.double))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(31):
        for batch in range(n_batch):
            head = 0
            batch_xs = Train_data[head:head+batch_size]
            batch_ys =  Train_label[head:head+batch_size]
            head = head + batch_size
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        test_acc = sess.run(accuracy,feed_dict={x:Test_data,y:Test_label,keep_prob:1.0})
        print("Iter " + str(epoch) + ",Testing Accuracy: " + str(test_acc))